In [1]:
from camera import Camera
import time


camera = Camera(2)
time.sleep(1) #
start = time.time()
combined_pointcloud = camera.combined_pointcloud()
end = time.time()
print(f'total time is: {end-start}')

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
total time is: 21.02886438369751


In [3]:
type(combined_pointcloud)

open3d.cpu.pybind.geometry.PointCloud

In [9]:
import numpy as np
from std_msgs.msg import Header
import rospy
import sensor_msgs.point_cloud2 as pc2
from sensor_msgs.msg import PointField

FIELDS_XYZ = [
    PointField(name='x', offset=0, datatype=PointField.FLOAT32, count=1),
    PointField(name='y', offset=4, datatype=PointField.FLOAT32, count=1),
    PointField(name='z', offset=8, datatype=PointField.FLOAT32, count=1),
]
FIELDS_XYZRGB = FIELDS_XYZ + \
    [PointField(name='rgb', offset=12, datatype=PointField.FLOAT32, count=1)]

# Bit operations
BIT_MOVE_16 = 2**16
BIT_MOVE_8 = 2**8

In [10]:
# Convert the datatype of point cloud from Open3D to ROS PointCloud2 (XYZRGB only)
def convertCloudFromOpen3dToRos(open3d_cloud, frame_id="map"):
    # Set "header"
    header = Header()
    header.stamp = rospy.Time.now()
    header.frame_id = frame_id

    # Set "fields" and "cloud_data"
    points=np.asarray(open3d_cloud.points)
    if not open3d_cloud.colors: # XYZ only
        fields=FIELDS_XYZ
        cloud_data=points
    else: # XYZ + RGB
        fields=FIELDS_XYZRGB
        # -- Change rgb color from "three float" to "one 24-byte int"
        # 0x00FFFFFF is white, 0x00000000 is black.
        colors = np.floor(np.asarray(open3d_cloud.colors)*255)
        colors = colors.astype(np.uint32)
        colors = colors[:,2] * BIT_MOVE_16 +colors[:,1] * BIT_MOVE_8 + colors[:,1]  
        colors = colors.view(np.float32)
        cloud_data = [tuple((*p, c)) for p, c in zip(points, colors)]
    # create ros_cloud
    return pc2.create_cloud(header, fields, cloud_data)

In [11]:
rospy.init_node('test')
output = convertCloudFromOpen3dToRos(combined_pointcloud, 'base_link')

In [1]:
from main import FirstDemo
grasper = FirstDemo()


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[INFO] [1719544418.006713]: Using robot_name kinova_gen3_lite , robot has 6 degrees of freedom and is_gripper_present is True
[INFO] [1719544418.017489]: Activating the action notifications...
[INFO] [1719544418.040405]: Successfully activated the Action Notifications!
[INFO] [1719544419.042615]: ------- You are now in Real Robot Case -------


In [2]:
grasper.main()

TypeError: Invalid number of arguments, args should be ['header', 'height', 'width', 'fields', 'is_bigendian', 'point_step', 'row_step', 'data', 'is_dense'] args are(PointCloud with 16320 points.,)